In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/train_V2']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# FIRST-AUTHOR: remove path printing
# import os
# print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

In [2]:
data = pd.read_csv('./input/train_V2.scaled.csv')

data.loc[:5,]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875
5,ff79c12f326506,289a6836a88d27,bac52627a12114,0,0,100.00,1,1,0,44,...,0,0.0000,0,0.00,0,0,34.70,1,0,0.0370


**TLD;DR**

Each match may not follow the limit set my the mode e.g. there are groups of 7 in a squad match when the limit is group of 4.
There may also be missing groups for each match as numGroup does not equal to maxPlace.

I want to take a look at an example solo match.

In [3]:
data_solo_example = data.loc[data.matchId == data.loc[data.matchType=='solo','matchId'].iloc[0],:].sort_values(by='winPlacePerc',ascending=False)
print(data_solo_example)

print('This example has {} players.'.format(len(data_solo_example.Id)))
print('This example has {} groups.'.format(len(data_solo_example.groupId.unique())))


                     Id         groupId         matchId  assists  boosts  \
2465259  2e7f7d12128f8b  c74517d95cfea3  ce9bc89b3ca08c        0       0   
1429646  4d384bc659275a  323a66b907a4bb  ce9bc89b3ca08c        0       5   
2084618  d88fb7e33a74c8  46ae53eeab70bf  ce9bc89b3ca08c        0       0   
3405449  bba14dfb77480b  1c558256143a48  ce9bc89b3ca08c        1       5   
3084080  6bde8e6e7f4044  a356d4ad03d323  ce9bc89b3ca08c        0       2   
...                 ...             ...             ...      ...     ...   
1491421  4f93811493f08a  da505f7b0a6c2e  ce9bc89b3ca08c        0       0   
4136389  52dd425f32895b  d197b657c27f7b  ce9bc89b3ca08c        0       0   
1437575  a424a153de8c9e  acfe066d986366  ce9bc89b3ca08c        0       0   
1983878  fc97f8036e98d0  acfe066d986366  ce9bc89b3ca08c        0       0   
2159449  7447cfdabe8eed  acfe066d986366  ce9bc89b3ca08c        0       0   

         damageDealt  DBNOs  headshotKills  heals  killPlace  ...  revives  \
2465259  

This solo match has 92 players but with 89 groups. This suggests that some players are in groups.

I went to investigate if the difference between consecutive winPlacePerc is consistent.

In [4]:
[x-data_solo_example.winPlacePerc.iloc[i+1] for i, x in enumerate(data_solo_example.winPlacePerc) if i < len(data_solo_example.winPlacePerc)-1]

[0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.02189999999999992,
 0.01100000000000001,
 0.01100000000000001,
 0.0,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01090000000000002,
 0.010999999999999899,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01090000000000002,
 0.01100000000000001,
 0.01100000000000001,
 0.010999999999999899,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01090000000000002,
 0.01100000000000001,
 0.01100000000000001,
 0.03299999999999992,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01090000000000002,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.01100000000000001,
 0.010999999999999954,
 0.01100000000000001,
 0.01100000000000001,
 

From the above calculation, I can see that it is not exactly consistent. 

I also found out that two players had the same winPlacePerc. Both players are also in the same team.

Let's a look at groupId count of more than 1.

In [5]:
data_solo_example.loc[data_solo_example.groupId.isin(
    list(data_solo_example.groupId.value_counts().loc[data_solo_example.groupId.value_counts()>1].index
        ))]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
1750121,27196a16bd93d5,add9daa3895f3d,ce9bc89b3ca08c,0,3,100.0,0,0,0,20,...,0,0.0,0,0.0,0,0,1811.000,6,0,0.9231
2434922,05e79262e0f4b7,add9daa3895f3d,ce9bc89b3ca08c,1,4,245.5,0,0,1,21,...,0,0.0,0,0.0,0,0,1627.000,8,0,0.9231
1437575,a424a153de8c9e,acfe066d986366,ce9bc89b3ca08c,0,0,0.0,0,0,0,91,...,0,0.0,0,0.0,0,0,0.000,0,0,0.0000
1983878,fc97f8036e98d0,acfe066d986366,ce9bc89b3ca08c,0,0,0.0,0,0,0,90,...,0,0.0,0,0.0,0,0,0.000,0,0,0.0000
2159449,7447cfdabe8eed,acfe066d986366,ce9bc89b3ca08c,0,0,0.0,0,0,0,92,...,0,0.0,0,0.0,0,0,7.559,1,0,0.0000


The table shows that these players seems to be unique. Therefore, there might have been data collection errors or cheating involved.

In [6]:
data_duo_example = data.loc[data.matchId == data.loc[data.matchType=='duo','matchId'].iloc[0],:].sort_values(by='winPlacePerc',ascending=False)
print(data_duo_example)

print('This example has {} players.'.format(len(data_duo_example.Id)))
print('This example has {} groups.'.format(len(data_duo_example.groupId.unique())))


                     Id         groupId         matchId  assists  boosts  \
1233239  e0ed1da68e2570  f2894b0e339e64  110163d8bb94ae        2       9   
3250980  4ef8a0e1cd21eb  f2894b0e339e64  110163d8bb94ae        1       2   
4112961  40fd2f897d286c  2ed8db3197175e  110163d8bb94ae        0       2   
3919736  977e8babbf99a3  2ed8db3197175e  110163d8bb94ae        0       4   
2635921  5815994c231aa2  07efef6f744e7b  110163d8bb94ae        0       2   
...                 ...             ...             ...      ...     ...   
2638659  b1df894f1e4117  4556f5553170dc  110163d8bb94ae        0       0   
283722   620c081cc47a70  4556f5553170dc  110163d8bb94ae        0       0   
4321453  b9d353ae92bfd5  4556f5553170dc  110163d8bb94ae        0       0   
217912   5baef877aa4123  4556f5553170dc  110163d8bb94ae        0       0   
1037862  ec7754aeb2e33e  4556f5553170dc  110163d8bb94ae        0       0   

         damageDealt  DBNOs  headshotKills  heals  killPlace  ...  revives  \
1233239  

In [7]:
data_duo_example.maxPlace

1233239    50
3250980    50
4112961    50
3919736    50
2635921    50
           ..
2638659    50
283722     50
4321453    50
217912     50
1037862    50
Name: maxPlace, Length: 98, dtype: int64

This duo example match shows that maxPlace need not be the same as numGroup.

Again, there are groups with number of players exceeding the duo limit.

In [8]:
data_duo_example.loc[data_duo_example.groupId.isin(
    list(data_duo_example.groupId.value_counts().loc[data_duo_example.groupId.value_counts()>2].index
        ))]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
797085,bfe94b51d3fa4a,39c559c4e78fe9,110163d8bb94ae,0,0,0.00,0,0,0,64,...,0,0.0,0,0.0,0,0,419.100,2,0,0.3878
296059,6cdc15670b2a19,39c559c4e78fe9,110163d8bb94ae,1,0,94.87,0,0,0,63,...,0,0.0,0,0.0,0,0,345.400,2,0,0.3878
1933250,dc39e2a45ccf5b,39c559c4e78fe9,110163d8bb94ae,0,0,185.00,1,1,0,22,...,0,0.0,0,0.0,0,0,219.100,4,0,0.3878
2353485,d789e3cd89e5c4,94d4c5c1b4be73,110163d8bb94ae,1,0,360.80,2,0,0,23,...,0,0.0,0,0.0,0,0,304.400,2,0,0.2857
3421019,76f4e45a6fe75c,94d4c5c1b4be73,110163d8bb94ae,0,0,114.00,2,0,0,24,...,0,0.0,0,0.0,0,0,140.400,3,0,0.2857
4389683,348353ec38918c,94d4c5c1b4be73,110163d8bb94ae,0,1,0.00,0,0,0,67,...,0,0.0,0,0.0,0,0,360.400,5,0,0.2857
3697866,f5d5ce53a6b4b7,7ea71c1a03d9f1,110163d8bb94ae,0,0,74.80,0,0,0,71,...,0,0.0,0,0.0,0,0,142.900,1,0,0.2245
1678505,e61e7a516cfc05,7ea71c1a03d9f1,110163d8bb94ae,0,0,0.00,0,0,0,69,...,0,0.0,0,0.0,0,0,191.900,3,0,0.2245
3596442,e2cf913c8f5456,7ea71c1a03d9f1,110163d8bb94ae,0,0,70.54,0,0,0,70,...,0,0.0,0,0.0,0,0,128.800,3,0,0.2245
433258,2d172f8e329e2f,e940798e572c86,110163d8bb94ae,0,0,54.18,0,0,0,79,...,0,0.0,0,0.0,0,0,137.000,2,0,0.1224


Let's take a look at an example squad match.

In [9]:
data_squad_example = data.loc[data.matchId == data.loc[data.matchType=='squad','matchId'].iloc[0],:].sort_values(by='winPlacePerc',ascending=False)
print(data_squad_example)

print('This example has {} players.'.format(len(data_squad_example.Id)))
print('This example has {} groups.'.format(len(data_squad_example.groupId.unique())))


                     Id         groupId         matchId  assists  boosts  \
1672444  63c0cc874a5949  4845feb4e6ec83  df014fbee741c6        1       2   
325696   e94d732a866185  4845feb4e6ec83  df014fbee741c6        1       0   
3854301  980612e924db01  4845feb4e6ec83  df014fbee741c6        1       2   
3414772  97e773463f3021  4845feb4e6ec83  df014fbee741c6        0       0   
2333073  24cf6e6dd055b1  ffca439de16174  df014fbee741c6        1       3   
...                 ...             ...             ...      ...     ...   
1306623  2e993b9758ad3a  bc5eaa8d8257fd  df014fbee741c6        0       0   
2501025  fa2994993e37b0  bc5eaa8d8257fd  df014fbee741c6        0       0   
906087   f0407397454aeb  bc5eaa8d8257fd  df014fbee741c6        0       0   
2766725  9329bbc182cdf1  bc5eaa8d8257fd  df014fbee741c6        0       0   
2799512  9cfe32bb342f7c  bc5eaa8d8257fd  df014fbee741c6        0       0   

         damageDealt  DBNOs  headshotKills  heals  killPlace  ...  revives  \
1672444  

In [10]:
data_squad_example.maxPlace

1672444    28
325696     28
3854301    28
3414772    28
2333073    28
           ..
1306623    28
2501025    28
906087     28
2766725    28
2799512    28
Name: maxPlace, Length: 97, dtype: int64

This squad example match also shows that maxPlace need not be the same as numGroup.

Again, there are groups with number of players exceeding the squad limit.

In [11]:
data_squad_example.loc[data_squad_example.groupId.isin(
    list(data_squad_example.groupId.value_counts().loc[data_squad_example.groupId.value_counts()>4].index
        ))]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
3443795,4217f8fe4e14ef,6453ad239d78fa,df014fbee741c6,0,0,200.00,2,0,0,21,...,0,0.0,0,0.0,0,0,51.480,3,0,0.1481
3388268,a70a780b9af32e,6453ad239d78fa,df014fbee741c6,0,0,184.70,0,0,0,34,...,3,0.0,0,0.0,0,0,137.200,3,0,0.1481
2169743,17dde20af451df,6453ad239d78fa,df014fbee741c6,0,0,295.00,2,0,0,20,...,0,0.0,0,0.0,0,0,125.200,3,0,0.1481
3946285,820ed073c1ff1d,6453ad239d78fa,df014fbee741c6,0,1,0.00,0,0,0,82,...,0,0.0,0,0.0,0,0,101.800,2,0,0.1481
1753465,8a8d4b236c58b1,6453ad239d78fa,df014fbee741c6,0,0,300.00,3,1,0,11,...,0,0.0,0,0.0,0,0,55.680,3,0,0.1481
2546138,d40106799e9780,6453ad239d78fa,df014fbee741c6,0,0,90.89,0,0,0,79,...,2,0.0,0,0.0,0,0,148.100,2,0,0.1481
3994880,6bbb9bf2faa466,6453ad239d78fa,df014fbee741c6,0,2,144.90,3,0,1,81,...,0,0.0,0,0.0,0,0,151.200,3,0,0.1481
2718243,7fde4f84c4018d,6453ad239d78fa,df014fbee741c6,0,0,0.00,0,0,0,83,...,0,0.0,0,0.0,0,0,152.500,6,0,0.1481
2980601,64ca1ae36bf45d,6453ad239d78fa,df014fbee741c6,0,0,161.90,1,0,0,35,...,0,0.0,0,0.0,0,0,205.400,4,0,0.1481
851321,fb05a3b360ef1d,6453ad239d78fa,df014fbee741c6,0,0,0.00,0,0,1,80,...,0,0.0,0,0.0,0,0,160.700,2,0,0.1481


**Conclusion**

The dataset shows that not all matches are following the limits of group size for each match. Also there may have been missing groups in each match as numGroup does not equal to maxPlace.
